# Deep Learning Assignment3: Batch Norm

## Author : Xiao Jing j655

## 0. Setup

In [1]:
# Import dependencies
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import numpy as np

False

In [57]:
# Set up your device 
cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if cuda else "cpu")

In [58]:
# Set up random seed to 1008. Do not change the random seed.
seed = 1008
torch.manual_seed(seed)
if cuda:
    torch.cuda.manual_seed_all(seed)

## 1. Data: MNIST
#### Load the MNIST training and test dataset using $\texttt{torch.utils.data.DataLoader}$ and $\texttt{torchvision.datasets}$. 

Hint: You might find Alf's notebook useful: https://github.com/Atcold/pytorch-Deep-Learning-Minicourse/blob/master/06-convnet.ipynb

### 1.1. Load Training Set [4 pts]

In [60]:
# Load the MNIST training set with batch size 128, apply data shuffling and normalization


train_loader = DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=128, shuffle=True)


In [81]:
len(train_loader)

469

### 1.1. Load Test Set [4 pts]

In [61]:
# Load the MNIST test set with batch size 128, apply data shuffling and normalization
test_loader = DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=128, shuffle=True)

## 2. Models
#### You are going to define two convolutional neural networks which are trained to classify MNIST digits

### 2.1. CNN without Batch Norm [15 pts]

In [62]:
# Fill in the values below that make this network valid for MNIST data

conv1_in_ch = 1
conv2_in_ch = 20
fc1_in_features =  4*4*50
fc2_in_features = 500
n_classes = 10

In [64]:
class NetWithoutBatchNorm(nn.Module):
    def __init__(self):
        super(NetWithoutBatchNorm, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=conv1_in_ch, out_channels=20, kernel_size=5, stride=1)
        self.conv2 = nn.Conv2d(in_channels=conv2_in_ch, out_channels=50, kernel_size=5, stride=1)
        self.fc1 = nn.Linear(in_features=fc1_in_features, out_features=500)
        self.fc2 = nn.Linear(in_features=fc2_in_features, out_features=n_classes)

    def forward(self, x): #28*28*1
        x = F.relu(self.conv1(x))#24*24*20
        x = F.max_pool2d(x, kernel_size=2, stride=2) #12*12*20
        x = F.relu(self.conv2(x)) # 8*8*50
        x = F.max_pool2d(x, kernel_size=2, stride=2) # 4*4*50
        x = x.view(-1, fc1_in_features) # reshaping 
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        # Return the log_softmax of x.
        x = F.log_softmax(x, dim=1)
        return x

### 2.2. CNN with Batch Norm [15 pts]

In [90]:
# Fill in the values below that make this network valid for MNIST data

conv1_bn_size = 20
conv2_bn_size = 50
fc1_bn_size = 500

In [66]:
# Define the CNN with architecture explained in Part 2.2
class NetWithBatchNorm(nn.Module):
    def __init__(self):
        super(NetWithBatchNorm, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=conv1_in_ch, out_channels=20, kernel_size=5, stride=1)
        self.conv1_bn = nn.BatchNorm2d(conv1_bn_size)
        self.conv2 = nn.Conv2d(in_channels=conv2_in_ch, out_channels=50, kernel_size=5, stride=1)
        self.conv2_bn = nn.BatchNorm2d(conv2_bn_size)
        self.fc1 = nn.Linear(in_features=fc1_in_features, out_features=500)
        self.fc1_bn = nn.BatchNorm1d(fc1_bn_size)
        self.fc2 = nn.Linear(in_features=fc2_in_features, out_features=n_classes)

    def forward(self, x):
        x = F.relu(self.conv1_bn(self.conv1(x)))
        x = F.max_pool2d(x, kernel_size=2, stride=2)
        x = F.relu(self.conv2_bn(self.conv2(x)))
        x = F.max_pool2d(x, kernel_size=2, stride=2)
        x = x.view(-1, fc1_in_features)
        x = F.relu(self.fc1_bn(self.fc1(x)))
        x = self.fc2(x)
        # Return the log_softmax of x.
        x = F.log_softmax(x, dim=1)
        return x

## 3. Training & Evaluation

### 3.1. Define training method [15 pts]

In [67]:
def train(model, device, train_loader, optimizer, epoch, log_interval = 100,perm=torch.arange(0, 784).long()):
    # Set model to training mode
    model.train()
    # Loop through data points
    for batch_idx, (data, target) in enumerate(train_loader):
        #pass # remove once implemented
    
        # Send data and target to device  
        # TODO
        model.to(device)
        
        # Zero out the optimizer
        # TODO
        optimizer.zero_grad()
        
        
        # Pass data through model
        # TODO
        output = model(data)
        
        
        # Compute the negative log likelihood loss
        # TODO
        loss = F.nll_loss(output, target)
        
        # Backpropagate loss
        # TODO
        loss.backward()
        
        # Make a step with the optimizer
        # TODO
        optimizer.step()
        # Print loss (uncomment lines below once implemented)
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                batch_indx, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

### 3.2. Define test method [15 pts]

In [85]:
# Define test method
def test(model, device, test_loader,perm=torch.arange(0, 784).long()):
    # Set model to evaluation mode
    model.eval()
    # Variable for the total loss 
    test_loss = 0
    # Counter for the correct predictions
    num_correct = 0
    
    with torch.no_grad():
        # Loop through data points
        for data, target in test_loader:
            #pass # remove once implemented
        
            # Send data to device            
            # TODO
            model.to(device)
            
            # Pass data through model
            # TODO 
            output = model(data)
            # Compute the negative log likelihood loss with reduction='sum' and add to total test_loss
            # TODO
            test_loss += F.nll_loss(output, target, reduction='sum').item()
            # Get predictions from the model for each data point
            # TODO 
            pred = output.data.max(1, keepdim=True)[1]
            # Add number of correct predictions to total num_correct 
            # TODO
            num_correct += pred.eq(target.data.view_as(pred)).cpu().sum().item()
    # Compute the average test_loss
    avg_test_loss  = test_loss / len(test_loader.dataset)
    accuracy = 100. *  num_correct / len(test_loader.dataset)
    # Print loss (uncomment lines below once implemented)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        avg_test_loss, num_correct, len(test_loader.dataset),
        accuracy))

### 3.3 Train NetWithoutBatchNorm() [5 pts]

In [86]:

# Deifne model and sent to device
model = NetWithoutBatchNorm()
#?????????????????????????????????sent to device

# Optimizer: SGD with learning rate of 1e-2 and momentum of 0.5
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

# Training loop with 10 epochs
for epoch in range(1, 10 + 1):
    #pass # remove once implemented
    # Train model
    train(model, device, train_loader, optimizer, epoch)
    # Test model
    test(model,device, test_loader)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.310994
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.478528
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.234608
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.274964
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.307261

Test set: Average loss: 0.1774, Accuracy: 9447/10000 (94%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.148213
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.165623
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.197644
Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.248426
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.135295

Test set: Average loss: 0.1194, Accuracy: 9634/10000 (96%)

Train Epoch: 3 [0/60000 (0%)]	Loss: 0.101527
Train Epoch: 3 [12800/60000 (21%)]	Loss: 0.123529
Train Epoch: 3 [25600/60000 (43%)]	Loss: 0.095013
Train Epoch: 3 [38400/60000 (64%)]	Loss: 0.046872
Train Epoch: 3 [51200/60000 (85%)]	Loss: 0.090121

Test set: Average loss: 0.0720, Accuracy: 9781/10000 (98%)

Train Epoch: 4 [0/60000 (0%)]	Loss: 0.126439
Train Epoch: 4 [12800/60000 (21%)]	Lo

### 3.4 Train NetWithBatchNorm() [5 pts]

In [91]:
# Deifne model and sent to device
model = NetWithBatchNorm()
#sent to device
model.to(device)
# Optimizer: SGD with learning rate of 1e-2 and momentum of 0.5
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)


# Training loop with 10 epochs
for epoch in range(1, 10 + 1):
    pass # remove once implemented
    
    # Train model
    train(model, device, train_loader, optimizer, epoch)
    
    # Test model
    test(model,device, test_loader)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.343081
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.406109
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.200201
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.124634
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.084208

Test set: Average loss: 0.1072, Accuracy: 9764/10000 (98%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.135641
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.140672
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.071285
Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.111352
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.075567

Test set: Average loss: 0.0677, Accuracy: 9836/10000 (98%)

Train Epoch: 3 [0/60000 (0%)]	Loss: 0.093614
Train Epoch: 3 [12800/60000 (21%)]	Loss: 0.079756
Train Epoch: 3 [25600/60000 (43%)]	Loss: 0.054882
Train Epoch: 3 [38400/60000 (64%)]	Loss: 0.050990
Train Epoch: 3 [51200/60000 (85%)]	Loss: 0.044233

Test set: Average loss: 0.0539, Accuracy: 9858/10000 (99%)

Train Epoch: 4 [0/60000 (0%)]	Loss: 0.041288
Train Epoch: 4 [12800/60000 (21%)]	Lo

## 4. Empirically, which of the models achieves higher accuracy faster? [2 pts]

Answer:  The model with Batch Norm seems run faster to achieve higher accuracy.